# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import datetime
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
model_revision='b098244'                  # 模型版本
training_tag=f"chatglm3-6b-epoch3-20240731_141714" # 微调后模型ID
peft_model_path = f"models/{training_tag}"  # 定义微调后的模型保存路径

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.bfloat16)

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision=model_revision)
base_model.requires_grad_(False)
base_model.eval()

/home/wood/miniconda3/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:49<00:00,  7.04s/it]


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [3]:
peft_model = PeftModel.from_pretrained(base_model, peft_model_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True,revision=model_revision)
def compare_results(query, base_model, peft_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)
    
    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = peft_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")

In [4]:
compare_results("解释下乾卦是什么？", base_model, peft_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由六个阳爻组成，预示着大吉大利。在这个卦中，大吉的意思和大吉大宜相同，都表示吉利的意思。君子们可以在君子之交中得到吉利的享受，而在小人的眼中，这种吉利享受会被破坏。乾卦的核心哲学是：君子观此卦象，可以得到大吉大利，但需tight-fist tight-mouth，慎行慢行，君子自交。', 'content': '\n乾卦象征天，代表着太阳刚刚升起，光明还在初升时的景象。因此，乾卦象征着君子观此卦象，可得到大吉大利。但必须紧握机会，谨慎行事，才能获得大吉大利。\n\n乾卦的核心哲学是：大吉，大吉，君子得喜。君子观此卦象，可以得到大吉，即享受吉利享受。然而，在小人的眼中，这种吉利享受会被破坏。因此，乾卦也象征着太阳初升时，光明尚未来临，君子需耐心等待，严于律己，最终可获得大吉。'}


微调后（chatglm3-6b-epoch3-20240731_141714）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [5]:
compare_results("周易中的讼卦是什么", base_model, peft_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（chatglm3-6b-epoch3-20240731_141714）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。坎卦代表着云，象征着天降，而乾卦代表着干，象征着天干。这个卦象预示着天降之象，但需要等待一段时间才能实现。

在这个卦象中，君子代表着上帝，君子之道在于等待时机的到来，以便天降之象实现。君子应时而行，应势而动， otherwise将会招致不幸。

讼卦的核心哲学是：止于至善，等待时机，顺其自然，勿急勿忙。君子应时而行，应势而动，止于至善，勿勿待时，等待天降之象。

讼卦的运势走向：先顺遂他人的意愿，再按计划行事，可得到好的结果。但需注意，不可过于急躁，必须耐心等待时机。

讼卦的运势解释：这是一个充满警示的卦象，提醒时要警惕可能出现的不利因素，并及时采取措施防范。君子应时而行，应势而动，止于至善，切勿勿待时，等待天降之象。


In [6]:
compare_results("师卦是什么？", base_model, peft_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '占卜是一种古老的预测方法，起源于我国。它是一种独特的卜卦术，主要通过观察卦象来推测未来的吉凶。在卜卦中，每卦都有不同的卦象，每种卦象都有其特殊的意义。师卦是其中的一种卦象，由坎卦（水）和坤卦（地）相叠而成。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。', 'content': '\n师卦的核心哲学是：顺水行舟，水落舟起，先声夺人，后明折尽。地生水， emotionally speaking，象征着顺从和合作。在这种情况下，人们应当遵循道德规范，注重合作，顺应自然，才能获得好运。\n\n师卦的意义不仅局限于卜卦，还涉及到人们日常生活中的各种情况。它提醒我们，在顺境中要谨慎行事，保持谦虚，尊重他人，才能一直好运。在逆境中，要坚定信念，勇敢面对困难，相信会有转机。\n\n师卦是一种吉卦，预示着顺境中的吉祥无灾。然而，它也提示着需要注意的是，在顺境中要谨慎行事，保持谦虚，尊重他人，才能一直好运。'}


微调后（chatglm3-6b-epoch3-20240731_141714）：
[gMASK]sop 师卦是什么？ 师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。在周易中，这个卦象征着军队的力量和军情的总指挥。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。师卦鼓励人们坚定正道，勤奋努力，尽管困难重重，仍需勇敢前进。
